In [25]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
from scipy import stats
import torch

from common import (
    load_x264,
    split_data,
    split_data_cv,
    evaluate_ii,
    evaluate_cc,
    prepare_result_df,
)

In [26]:
## Configuration
random_seed = 33154

# Enter names of performance columns to consider
performances = ["rel_kbs"]

# Number of nearest neighbours to consider
# Make multiples to allow better budget comparison
topk_values = (1, 3, 5, 15, 25)
topr_values = (1, 3, 5, 15, 25)

data_dir = "../data"

In [27]:
## Load and prepare data
perf_matrix, input_features, config_features, all_performances = load_x264(
    data_dir=data_dir
)

print(f"Loaded data x264")
print(f"perf_matrix:{perf_matrix.shape}")
print(f"input_features:{input_features.shape}")
print(f"config_features:{config_features.shape}")

data_split = split_data(perf_matrix, random_state=random_seed)
train_inp = data_split["train_inp"]
train_cfg = data_split["train_cfg"]
test_inp = data_split["test_inp"]
test_cfg = data_split["test_cfg"]

# This is a look up for performance measurements from inputname + configurationID
input_config_map = (
    perf_matrix[["inputname", "configurationID"] + performances]
    .sort_values(["inputname", "configurationID"])
    .set_index(["inputname", "configurationID"])
)
all_input_names = pd.Series(
    input_config_map.index.get_level_values("inputname").unique()
)
all_config_ids = pd.Series(
    input_config_map.index.get_level_values("configurationID").unique()
)

regret_map = input_config_map.groupby("inputname").transform(
    lambda x: (x - x.min()).abs() / abs(x.min())
)
average_mape = regret_map.mean(axis=1)

rank_map = input_config_map.groupby("inputname").transform(
    lambda x: stats.rankdata(x, method="min")
)
average_ranks = rank_map.mean(axis=1)

Loaded data x264
perf_matrix:(258687, 45)
input_features:(1287, 21)
config_features:(201, 48)
Training data: 63.64%
Both new: 4.09%
Config new: 16.31%
Input new: 15.96%


In [28]:
## Prepare necessary torch tensors
# Prepare lookup tables for input/configuration performances as torch tensors
rank_arr = torch.from_numpy(
    rank_map.reset_index()  # .loc[(train_inp, train_cfg), :]
    .pivot_table(index="inputname", columns="configurationID", values=performances[0])
    .values
)
regret_arr = torch.from_numpy(
    regret_map.reset_index()  # .loc[(train_inp, train_cfg), :]
    .pivot_table(index="inputname", columns="configurationID", values=performances[0])
    .values
)

# Prepare and select training/test data according to random split
input_arr = torch.from_numpy(input_features.values).float()
config_arr = torch.from_numpy(config_features.values).float()

train_input_mask = input_features.index.isin(train_inp)
test_input_mask = input_features.index.isin(test_inp)

train_config_mask = config_features.index.isin(train_cfg)
test_config_mask = config_features.index.isin(test_cfg)

train_input_arr = input_arr[train_input_mask]
train_config_arr = config_arr[train_config_mask]

In [29]:
train_cc_rank = []
train_cc_ratio = []
train_cc_regret = []

test_cc_rank = []
test_cc_ratio = []
test_cc_regret = []

train_ii_rank = []
train_ii_ratio = []
train_ii_regret = []

test_ii_rank = []
test_ii_ratio = []
test_ii_regret = []

# Query: test data
# Database: train data

for topk in topk_values:
    train_cc = evaluate_cc(
        config_arr,
        rank_arr=rank_arr,
        regret_arr=regret_arr,
        n_neighbors=topk,
        n_recs=topr_values,
        query_mask=torch.from_numpy(train_config_mask),
        reference_mask=torch.from_numpy(train_config_mask),
    )
    train_cc_rank.append(train_cc[0].numpy())
    train_cc_regret.append(train_cc[1].numpy())
    train_cc_ratio.append(train_cc[2].numpy())

    test_cc = evaluate_cc(
        config_arr,
        rank_arr=rank_arr,
        regret_arr=regret_arr,
        n_neighbors=topk,
        n_recs=topr_values,
        query_mask=torch.from_numpy(test_config_mask),
        reference_mask=torch.from_numpy(train_config_mask),
    )
    test_cc_rank.append(test_cc[0].numpy())
    test_cc_regret.append(test_cc[1].numpy())
    test_cc_ratio.append(test_cc[2].numpy())

    train_ii = evaluate_ii(
        input_arr,
        rank_arr=rank_arr,
        regret_arr=regret_arr,
        n_neighbors=topk,
        n_recs=topr_values,
        query_mask=torch.from_numpy(train_input_mask),
        reference_mask=torch.from_numpy(train_input_mask),
    )
    train_ii_rank.append(train_ii[0].numpy())
    train_ii_regret.append(train_ii[1].numpy())
    train_ii_ratio.append(train_ii[2].numpy())

    test_ii = evaluate_ii(
        input_arr,
        rank_arr=rank_arr,
        regret_arr=regret_arr,
        n_neighbors=topk,
        n_recs=topr_values,
        query_mask=torch.from_numpy(test_input_mask),
        reference_mask=torch.from_numpy(train_input_mask),
    )
    test_ii_rank.append(test_ii[0].numpy())
    test_ii_regret.append(test_ii[1].numpy())
    test_ii_ratio.append(test_ii[2].numpy())

In [30]:
# TODO Share results in README

print(
    "train cc ratio\n",
    prepare_result_df(train_cc_ratio, topr_values, topk_values),
    "\n",
)
print(
    "train cc best rank\n",
    prepare_result_df(train_cc_rank, topr_values, topk_values),
    "\n",
)
print(
    "train cc best regret\n",
    prepare_result_df(train_cc_regret, topr_values, topk_values),
    "\n",
)

print(
    "test cc ratio\n", prepare_result_df(test_cc_ratio, topr_values, topk_values), "\n"
)
print(
    "test cc best rank\n",
    prepare_result_df(test_cc_rank, topr_values, topk_values),
    "\n",
)
print(
    "test cc best regret\n",
    prepare_result_df(test_cc_regret, topr_values, topk_values),
    "\n",
)

print(
    "train ii ratio\n",
    prepare_result_df(train_ii_ratio, topr_values, topk_values),
    "\n",
)
print(
    "train ii best rank\n",
    prepare_result_df(train_ii_rank, topr_values, topk_values),
    "\n",
)
print(
    "train ii best regret\n",
    prepare_result_df(train_ii_regret, topr_values, topk_values),
    "\n",
)

print(
    "test ii ratio\n", prepare_result_df(test_ii_ratio, topr_values, topk_values), "\n"
)
print(
    "test ii best rank\n",
    prepare_result_df(test_ii_rank, topr_values, topk_values),
    "\n",
)
print(
    "test ii best regret\n",
    prepare_result_df(test_ii_regret, topr_values, topk_values),
    "\n",
)

train cc ratio
             r                                            
            1          3          5         15         25
k                                                        
1         NaN        NaN        NaN        NaN        NaN
3   19.375002  27.291668  29.624998  35.083336  37.462502
5   20.781248  29.531252  32.062500  39.135414  42.356255
15  24.776787  34.479164  38.116074  45.648808  50.198215
25  26.119793  37.343754  40.838539  49.336803  54.812504 

train cc best rank
            r                                        
           1         3         5        15        25
k                                                   
1   7.099359  5.805653  5.145202  3.775738  3.402292
3   5.154429  3.882576  3.291084  2.549048  2.287782
5   4.307498  3.177448  2.812257  2.143065  1.941045
15  2.573815  2.011461  1.737082  1.410742  1.360723
25  2.028458  1.492813  1.384033  1.254371  1.235431 

train cc best regret
             r                                     

In [31]:
# Run cross-validation over all train/test splits
dfs = []

for data_split in split_data_cv(perf_matrix, random_state=random_seed):
    train_inp = data_split["train_inp"]
    train_cfg = data_split["train_cfg"]
    test_inp = data_split["test_inp"]
    test_cfg = data_split["test_cfg"]

    # Prepare and select training/test data according to random split
    input_arr = torch.from_numpy(input_features.values).float()
    config_arr = torch.from_numpy(config_features.values).float()

    train_input_mask = input_features.index.isin(train_inp)
    test_input_mask = input_features.index.isin(test_inp)

    train_config_mask = config_features.index.isin(train_cfg)
    test_config_mask = config_features.index.isin(test_cfg)

    train_input_arr = input_arr[train_input_mask]
    train_config_arr = config_arr[train_config_mask]

    train_cc_rank = []
    train_cc_ratio = []
    train_cc_regret = []

    test_cc_rank = []
    test_cc_ratio = []
    test_cc_regret = []

    train_ii_rank = []
    train_ii_ratio = []
    train_ii_regret = []

    test_ii_rank = []
    test_ii_ratio = []
    test_ii_regret = []

    # Query: test data
    # Database: train data

    for topk in topk_values:
        train_cc = evaluate_cc(
            config_arr,
            rank_arr=rank_arr,
            regret_arr=regret_arr,
            n_neighbors=topk,
            n_recs=topr_values,
            query_mask=torch.from_numpy(train_config_mask),
            reference_mask=torch.from_numpy(train_config_mask),
        )
        train_cc_rank.append(train_cc[0].numpy())
        train_cc_regret.append(train_cc[1].numpy())
        train_cc_ratio.append(train_cc[2].numpy())

        test_cc = evaluate_cc(
            config_arr,
            rank_arr=rank_arr,
            regret_arr=regret_arr,
            n_neighbors=topk,
            n_recs=topr_values,
            query_mask=torch.from_numpy(test_config_mask),
            reference_mask=torch.from_numpy(train_config_mask),
        )
        test_cc_rank.append(test_cc[0].numpy())
        test_cc_regret.append(test_cc[1].numpy())
        test_cc_ratio.append(test_cc[2].numpy())

        train_ii = evaluate_ii(
            input_arr,
            rank_arr=rank_arr,
            regret_arr=regret_arr,
            n_neighbors=topk,
            n_recs=topr_values,
            query_mask=torch.from_numpy(train_input_mask),
            reference_mask=torch.from_numpy(train_input_mask),
        )
        train_ii_rank.append(train_ii[0].numpy())
        train_ii_regret.append(train_ii[1].numpy())
        train_ii_ratio.append(train_ii[2].numpy())

        test_ii = evaluate_ii(
            input_arr,
            rank_arr=rank_arr,
            regret_arr=regret_arr,
            n_neighbors=topk,
            n_recs=topr_values,
            query_mask=torch.from_numpy(test_input_mask),
            reference_mask=torch.from_numpy(train_input_mask),
        )
        test_ii_rank.append(test_ii[0].numpy())
        test_ii_regret.append(test_ii[1].numpy())
        test_ii_ratio.append(test_ii[2].numpy())

    dfs.append(
        prepare_result_df(
            train_cc_rank,
            topr_values,
            topk_values,
            {"metric": "rank", "mode": "cc", "split": "train"},
        )
    )
    dfs.append(
        prepare_result_df(
            train_cc_regret,
            topr_values,
            topk_values,
            {"metric": "regret", "mode": "cc", "split": "train"},
        )
    )
    dfs.append(
        prepare_result_df(
            train_cc_ratio,
            topr_values,
            topk_values,
            {"metric": "ratio", "mode": "cc", "split": "train"},
        )
    )

    dfs.append(
        prepare_result_df(
            test_cc_rank,
            topr_values,
            topk_values,
            {"metric": "rank", "mode": "cc", "split": "test"},
        )
    )
    dfs.append(
        prepare_result_df(
            test_cc_regret,
            topr_values,
            topk_values,
            {"metric": "regret", "mode": "cc", "split": "test"},
        )
    )
    dfs.append(
        prepare_result_df(
            test_cc_ratio,
            topr_values,
            topk_values,
            {"metric": "ratio", "mode": "cc", "split": "test"},
        )
    )

    dfs.append(
        prepare_result_df(
            train_ii_rank,
            topr_values,
            topk_values,
            {"metric": "rank", "mode": "ii", "split": "train"},
        )
    )
    dfs.append(
        prepare_result_df(
            train_ii_regret,
            topr_values,
            topk_values,
            {"metric": "regret", "mode": "ii", "split": "train"},
        )
    )
    dfs.append(
        prepare_result_df(
            train_ii_ratio,
            topr_values,
            topk_values,
            {"metric": "ratio", "mode": "ii", "split": "train"},
        )
    )

    dfs.append(
        prepare_result_df(
            test_ii_rank,
            topr_values,
            topk_values,
            {"metric": "rank", "mode": "ii", "split": "test"},
        )
    )
    dfs.append(
        prepare_result_df(
            test_ii_regret,
            topr_values,
            topk_values,
            {"metric": "regret", "mode": "ii", "split": "test"},
        )
    )
    dfs.append(
        prepare_result_df(
            test_ii_ratio,
            topr_values,
            topk_values,
            {"metric": "ratio", "mode": "ii", "split": "test"},
        )
    )

full_df = pd.concat(dfs)
full_df.groupby(["mode", "split", "metric", "k"]).mean()

Training data: 55.96%
Both new: 6.35%
Config new: 19.02%
Input new: 18.67%
Training data: 56.33%
Both new: 6.22%
Config new: 18.65%
Input new: 18.80%
Training data: 56.33%
Both new: 6.22%
Config new: 18.65%
Input new: 18.80%
Training data: 56.39%
Both new: 6.20%
Config new: 18.67%
Input new: 18.74%


r                                            
                               1          3          5         15         25
mode split metric k                                                         
cc   test  rank   1    10.594002   9.280840   8.676280   7.647364   7.210522
                  3     9.048958   7.595121   7.035780   6.010794   5.631161
                  5     8.183449   6.798528   6.256806   5.032253   4.712868
                  15    6.457646   4.863301   4.384273   3.634482   3.341743
                  25    5.446082   3.865670   3.656543   3.170072   2.982358
           ratio  1          NaN        NaN        NaN        NaN        NaN
                  3    13.950981  23.746731  27.164707  32.930717  36.077255
                  5    14.931372  24.561275  28.040197  34.391994  38.031666
                  15   21.637955  32.811390  36.142296  44.255043  49.154198
                  25   23.133169  36.057053  39.528923  48.515224  54.401619
           regret 1   150.609314  87.177200  66.151222  49.709167  43.587460
                  3    82.385254  51.683174  46.881512  36.541340  33.697929
                  5    62.304375  43.110703  39.734547  31.884159  30.018906
                  15   40.204136  31.870258  30.308922  26.121069  24.423744
                  25   32.700115  27.509514  26.622166  23.745523  22.429277
     train rank   1     7.332560   5.936970   5.415041   4.053437   3.640843
                  3     5.462202   4.317197   3.760926   2.770912   2.452447
                  5     4.486753   3.402778   2.957452   2.261945   1.976724
                  15    2.598980   2.124932   1.836155   1.534679   1.449106
                  25    1.962165   1.650836   1.545661   1.363280   1.337892
           ratio  1          NaN        NaN        NaN        NaN        NaN
                  3    15.098785  22.947205  26.621080  32.349228  35.916622
                  5    16.137140  24.924208  28.280960  35.187935  39.182793
                  15   21.707348  32.602859  35.801468  43.769699  49.014633
                  25   22.696514  35.230453  38.695843  47.741020  53.777225
           regret 1    88.515274  46.238998  37.953270  25.982136  23.088804
                  3    49.040230  28.812748  25.405693  18.779388  17.148451
                  5    36.227837  22.669678  21.010050  16.343897  14.953991
                  15   19.076746  15.029872  14.182199  11.603624  10.798914
                  25   15.246889  12.382740  12.021671  10.224865   9.851536
ii   test  rank   1    26.054417  19.148766  16.198116  10.942026   7.113796
                  3    11.113079   7.404642   6.396124   3.628683   2.423609
                  5     6.829665   4.451962   3.636693   1.767563   1.234477
                  15    2.213184   1.188522   0.868064   0.365624   0.239627
                  25    1.307169   0.687545   0.479211   0.148821   0.082699
           ratio  1          NaN        NaN        NaN        NaN        NaN
                  3    23.661137  29.733807  33.116013  44.115948  53.065964
                  5    31.371056  38.208370  42.128441  53.778419  62.553452
                  15   50.771957  57.732582  61.901451  73.304230  80.168320
                  25   60.036480  66.058937  70.011703  80.541206  86.139771
           regret 1    43.920624  34.388096  29.705135  21.204117  15.203573
                  3    23.269501  16.420698  14.756729   8.799952   6.659415
                  5    16.036232  11.387281   9.473250   5.285122   4.150132
                  15    6.182018   3.676162   3.121352   1.594164   1.002506
                  25    3.993323   2.058841   1.576825   0.615925   0.321153
     train rank   1    25.258106  18.839020  16.013027  10.540405   6.600419
                  3    10.196865   6.837489   5.762649   3.350584   2.276775
                  5     6.304160   3.947913   3.195758   1.747497   1.181271
                  15    1.893320   1.061531   0.818230   0.420450   0.266090
                  25    1.1

In [32]:
# full_df.groupby(["mode", "split", "metric", "k"]).mean().to_clipboard()
dfmean = (
    full_df.reset_index()
    .groupby(["mode", "split", "metric", "k"], as_index=False)
    .mean()
)
dfmean.to_csv("knn_config_recommendation.csv")

/var/folders/rf/xz0f5yz96cb_qq_gcl42zqvw0000gn/T/ipykernel_47471/2273926967.py:5: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  .mean()


In [33]:
(dfmean["mode"] == "cc") & (dfmean["split"] == 'test') & (dfmean["metric"] == 'rank')

0      True
1      True
2      True
3      True
4      True
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
36    False
37    False
38    False
39    False
40    False
41    False
42    False
43    False
44    False
45    False
46    False
47    False
48    False
49    False
50    False
51    False
52    False
53    False
54    False
55    False
56    False
57    False
58    False
59    False
dtype: bool

In [34]:
print(
    dfmean[
        (dfmean["mode"] == "cc")
        & (dfmean["split"] == "test")
        & (dfmean["metric"] == "rank")
    ]
    .drop(columns=["mode", "split", "metric"])
    .to_latex(index=False, float_format="%.2f", caption="Rank")
)
print(
    dfmean[
        (dfmean["mode"] == "cc")
        & (dfmean["split"] == "test")
        & (dfmean["metric"] == "regret")
    ]
    .drop(columns=["mode", "split", "metric"])
    .to_latex(index=False, float_format="%.2f", caption="Regret")
)
print(
    dfmean[
        (dfmean["mode"] == "cc")
        & (dfmean["split"] == "test")
        & (dfmean["metric"] == "ratio")
    ]
    .drop(columns=["mode", "split", "metric"])
    .to_latex(index=False, float_format="%.2f", na_rep="-", caption="Ratio")
)

\begin{table}
\centering
\caption{Rank}
\begin{tabular}{rrrrrr}
\toprule
 k & \multicolumn{5}{l}{r} \\
   &     1 &    3 &    5 &   15 &   25 \\
\midrule
 1 & 10.59 & 9.28 & 8.68 & 7.65 & 7.21 \\
 3 &  9.05 & 7.60 & 7.04 & 6.01 & 5.63 \\
 5 &  8.18 & 6.80 & 6.26 & 5.03 & 4.71 \\
15 &  6.46 & 4.86 & 4.38 & 3.63 & 3.34 \\
25 &  5.45 & 3.87 & 3.66 & 3.17 & 2.98 \\
\bottomrule
\end{tabular}
\end{table}

\begin{table}
\centering
\caption{Regret}
\begin{tabular}{rrrrrr}
\toprule
 k & \multicolumn{5}{l}{r} \\
   &      1 &     3 &     5 &    15 &    25 \\
\midrule
 1 & 150.61 & 87.18 & 66.15 & 49.71 & 43.59 \\
 3 &  82.39 & 51.68 & 46.88 & 36.54 & 33.70 \\
 5 &  62.30 & 43.11 & 39.73 & 31.88 & 30.02 \\
15 &  40.20 & 31.87 & 30.31 & 26.12 & 24.42 \\
25 &  32.70 & 27.51 & 26.62 & 23.75 & 22.43 \\
\bottomrule
\end{tabular}
\end{table}

\begin{table}
\centering
\caption{Ratio}
\begin{tabular}{rrrrrr}
\toprule
 k & \multicolumn{5}{l}{r} \\
   &     1 &     3 &     5 &    15 &    25 \\
\midrule
 1 

/var/folders/rf/xz0f5yz96cb_qq_gcl42zqvw0000gn/T/ipykernel_47471/1447550786.py:7: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  .drop(columns=["mode", "split", "metric"])
/var/folders/rf/xz0f5yz96cb_qq_gcl42zqvw0000gn/T/ipykernel_47471/1447550786.py:8: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  .to_latex(index=False, float_format="%.2f", caption="Rank")
/var/folders/rf/xz0f5yz96cb_qq_gcl42zqvw0000gn/T/ipykernel_47471/1447550786.py:16: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  .drop(columns=["mode", "split", "metric"])
/var/folders/rf/xz0f5yz96cb_qq_gcl42zqvw0000gn/T/ipykernel_47471/1447550786.

In [35]:
print(
    dfmean[
        (dfmean["mode"] == "ii")
        & (dfmean["split"] == "test")
        & (dfmean["metric"] == "rank")
    ]
    .drop(columns=["mode", "split", "metric"])
    .to_latex(index=False, float_format="%.2f", caption="Rank")
)
print(
    dfmean[
        (dfmean["mode"] == "ii")
        & (dfmean["split"] == "test")
        & (dfmean["metric"] == "regret")
    ]
    .drop(columns=["mode", "split", "metric"])
    .to_latex(index=False, float_format="%.2f", caption="Regret")
)
print(
    dfmean[
        (dfmean["mode"] == "ii")
        & (dfmean["split"] == "test")
        & (dfmean["metric"] == "ratio")
    ]
    .drop(columns=["mode", "split", "metric"])
    .to_latex(index=False, float_format="%.2f", na_rep="-", caption="Ratio")
)

\begin{table}
\centering
\caption{Rank}
\begin{tabular}{rrrrrr}
\toprule
 k & \multicolumn{5}{l}{r} \\
   &     1 &     3 &     5 &    15 &   25 \\
\midrule
 1 & 26.05 & 19.15 & 16.20 & 10.94 & 7.11 \\
 3 & 11.11 &  7.40 &  6.40 &  3.63 & 2.42 \\
 5 &  6.83 &  4.45 &  3.64 &  1.77 & 1.23 \\
15 &  2.21 &  1.19 &  0.87 &  0.37 & 0.24 \\
25 &  1.31 &  0.69 &  0.48 &  0.15 & 0.08 \\
\bottomrule
\end{tabular}
\end{table}

\begin{table}
\centering
\caption{Regret}
\begin{tabular}{rrrrrr}
\toprule
 k & \multicolumn{5}{l}{r} \\
   &     1 &     3 &     5 &    15 &    25 \\
\midrule
 1 & 43.92 & 34.39 & 29.71 & 21.20 & 15.20 \\
 3 & 23.27 & 16.42 & 14.76 &  8.80 &  6.66 \\
 5 & 16.04 & 11.39 &  9.47 &  5.29 &  4.15 \\
15 &  6.18 &  3.68 &  3.12 &  1.59 &  1.00 \\
25 &  3.99 &  2.06 &  1.58 &  0.62 &  0.32 \\
\bottomrule
\end{tabular}
\end{table}

\begin{table}
\centering
\caption{Ratio}
\begin{tabular}{rrrrrr}
\toprule
 k & \multicolumn{5}{l}{r} \\
   &     1 &     3 &     5 &    15 &    25 \\


/var/folders/rf/xz0f5yz96cb_qq_gcl42zqvw0000gn/T/ipykernel_47471/1073859418.py:7: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  .drop(columns=["mode", "split", "metric"])
/var/folders/rf/xz0f5yz96cb_qq_gcl42zqvw0000gn/T/ipykernel_47471/1073859418.py:8: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  .to_latex(index=False, float_format="%.2f", caption="Rank")
/var/folders/rf/xz0f5yz96cb_qq_gcl42zqvw0000gn/T/ipykernel_47471/1073859418.py:16: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  .drop(columns=["mode", "split", "metric"])
/var/folders/rf/xz0f5yz96cb_qq_gcl42zqvw0000gn/T/ipykernel_47471/1073859418.

In [36]:
m = pd.concat(
    (
        dfmean[
            (dfmean["mode"] == "cc")
            & (dfmean["split"] == "test")
            & (dfmean["metric"] == "rank")
        ]
        .drop(columns=["mode", "split", "metric"])
        .set_index("k"),
        dfmean[
            (dfmean["mode"] == "cc")
            & (dfmean["split"] == "test")
            & (dfmean["metric"] == "ratio")
        ]
        .drop(columns=["mode", "split", "metric"])
        .set_index("k"),
        dfmean[
            (dfmean["mode"] == "cc")
            & (dfmean["split"] == "test")
            & (dfmean["metric"] == "regret")
        ]
        .drop(columns=["mode", "split", "metric"])
        .set_index("k"),
    ),
    axis=1,
    keys=["rank", "ratio", "regret"],
)
print(
    m.to_latex(
        index=True,
        float_format="%.2f",
        na_rep="-",
        caption="Configuration-Configuration",
    )
)

\begin{table}
\centering
\caption{Configuration-Configuration}
\begin{tabular}{lrrrrrrrrrrrrrrr}
\toprule
{} & \multicolumn{5}{l}{rank} & \multicolumn{5}{l}{ratio} & \multicolumn{5}{l}{regret} \\
{} & \multicolumn{5}{l}{r} & \multicolumn{5}{l}{r} & \multicolumn{5}{l}{r} \\
{} &     1 &    3 &    5 &   15 &   25 &     1 &     3 &     5 &    15 &    25 &      1 &     3 &     5 &    15 &    25 \\
k  &       &      &      &      &      &       &       &       &       &       &        &       &       &       &       \\
\midrule
1  & 10.59 & 9.28 & 8.68 & 7.65 & 7.21 &     - &     - &     - &     - &     - & 150.61 & 87.18 & 66.15 & 49.71 & 43.59 \\
3  &  9.05 & 7.60 & 7.04 & 6.01 & 5.63 & 13.95 & 23.75 & 27.16 & 32.93 & 36.08 &  82.39 & 51.68 & 46.88 & 36.54 & 33.70 \\
5  &  8.18 & 6.80 & 6.26 & 5.03 & 4.71 & 14.93 & 24.56 & 28.04 & 34.39 & 38.03 &  62.30 & 43.11 & 39.73 & 31.88 & 30.02 \\
15 &  6.46 & 4.86 & 4.38 & 3.63 & 3.34 & 21.64 & 32.81 & 36.14 & 44.26 & 49.15 &  40.20 & 31.87 & 30.3

/var/folders/rf/xz0f5yz96cb_qq_gcl42zqvw0000gn/T/ipykernel_47471/4141552973.py:8: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  .drop(columns=["mode", "split", "metric"])
/var/folders/rf/xz0f5yz96cb_qq_gcl42zqvw0000gn/T/ipykernel_47471/4141552973.py:15: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  .drop(columns=["mode", "split", "metric"])
/var/folders/rf/xz0f5yz96cb_qq_gcl42zqvw0000gn/T/ipykernel_47471/4141552973.py:22: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  .drop(columns=["mode", "split", "metric"])
/var/folders/rf/xz0f5yz96cb_qq_gcl42zqvw0000gn/T/ipykernel_47471/4141552973.py:29: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is

In [37]:
m = pd.concat(
    (
        dfmean[
            (dfmean["mode"] == "ii")
            & (dfmean["split"] == "test")
            & (dfmean["metric"] == "rank")
        ]
        .drop(columns=["mode", "split", "metric"])
        .set_index("k"),
        dfmean[
            (dfmean["mode"] == "ii")
            & (dfmean["split"] == "test")
            & (dfmean["metric"] == "ratio")
        ]
        .drop(columns=["mode", "split", "metric"])
        .set_index("k"),
        dfmean[
            (dfmean["mode"] == "ii")
            & (dfmean["split"] == "test")
            & (dfmean["metric"] == "regret")
        ]
        .drop(columns=["mode", "split", "metric"])
        .set_index("k"),
    ),
    axis=1,
    keys=["rank", "ratio", "regret"],
)
print(m.to_latex(index=True, float_format="%.2f", na_rep="-", caption="Input-Input"))

\begin{table}
\centering
\caption{Input-Input}
\begin{tabular}{lrrrrrrrrrrrrrrr}
\toprule
{} & \multicolumn{5}{l}{rank} & \multicolumn{5}{l}{ratio} & \multicolumn{5}{l}{regret} \\
{} & \multicolumn{5}{l}{r} & \multicolumn{5}{l}{r} & \multicolumn{5}{l}{r} \\
{} &     1 &     3 &     5 &    15 &   25 &     1 &     3 &     5 &    15 &    25 &      1 &     3 &     5 &    15 &    25 \\
k  &       &       &       &       &      &       &       &       &       &       &        &       &       &       &       \\
\midrule
1  & 26.05 & 19.15 & 16.20 & 10.94 & 7.11 &     - &     - &     - &     - &     - &  43.92 & 34.39 & 29.71 & 21.20 & 15.20 \\
3  & 11.11 &  7.40 &  6.40 &  3.63 & 2.42 & 23.66 & 29.73 & 33.12 & 44.12 & 53.07 &  23.27 & 16.42 & 14.76 &  8.80 &  6.66 \\
5  &  6.83 &  4.45 &  3.64 &  1.77 & 1.23 & 31.37 & 38.21 & 42.13 & 53.78 & 62.55 &  16.04 & 11.39 &  9.47 &  5.29 &  4.15 \\
15 &  2.21 &  1.19 &  0.87 &  0.37 & 0.24 & 50.77 & 57.73 & 61.90 & 73.30 & 80.17 &   6.18 &  3.68 &  3

/var/folders/rf/xz0f5yz96cb_qq_gcl42zqvw0000gn/T/ipykernel_47471/1315325197.py:8: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  .drop(columns=["mode", "split", "metric"])
/var/folders/rf/xz0f5yz96cb_qq_gcl42zqvw0000gn/T/ipykernel_47471/1315325197.py:15: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  .drop(columns=["mode", "split", "metric"])
/var/folders/rf/xz0f5yz96cb_qq_gcl42zqvw0000gn/T/ipykernel_47471/1315325197.py:22: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  .drop(columns=["mode", "split", "metric"])
/var/folders/rf/xz0f5yz96cb_qq_gcl42zqvw0000gn/T/ipykernel_47471/1315325197.py:28: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is